In [5]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO
import string

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text


In [6]:
def isAllCaps(line):
    for ch in line:
        if(ch.islower()):
            return False
    return True
def isAction(i,lines):
    #print("i-1,i-3=>",lines[i-1] , lines[i-2] , lines[i-3])
    if(lines[i] == ""):
        return False
    if(i+3>=len(lines)):
        return False
    if("--" in lines[i]):
        return False
    if lines[i][0].islower():
        return False
    if lines[i][-1] == '?':
        return False
    if(lines[i-1] == lines[i-2] == lines[i-3] == ""):
        if(lines[i+1] == lines[i+2] == lines[i+3] == ""):
            if(lines[i][1].islower()):
                if(lines[i-4] != "" and lines[i-4][0] == '('):
                    return False
                if(lines[i-4] == ""):
                    return True
                return True
    if "EXT." in lines[i-4] or "INT." in lines[i-4]:
        return True
    return False
def isName(name):
    if not isAllCaps(name):
        return False
    if "INT." in name or "EXT." in name:
        return False
    if len(name)<3:
        return False
    return True


In [8]:
my_text = convert_pdf_to_txt('FRANKENSTEIN_by_Max_Landis_2011_05_23.pdf')
lines = my_text.split('\n')
#print(lines)
txtfile = open("frompdfminer.txt","w")
#remove tabs,(MORE),pagenumbers from text
for i,line in enumerate(lines):
    txtfile.write(line+'\n')
        

In [4]:
#my_text = docx2txt.process("Boyhood-screenplay-11-14-FINAL.docx")
#lines = my_text.split('\n')
out = open("out.txt","w")
my_text = convert_pdf_to_txt('Boyhood-screenplay-11-14-FINAL.pdf')
lines = my_text.split('\n')
print(lines)
txtfile = open("boyhood_modified.txt","w")
discovery = 1
#remove tabs,(MORE),pagenumbers from text
for i,line in enumerate(lines):
    lines[i] = lines[i].replace('\t',' ')
    lines[i] = lines[i].replace('(MORE)','')
    if(2<=len(lines[i])<=5 and lines[i][-1] == '.' and  lines[i][:-1].isdigit()):
        #print("page:",lines[i])
        lines[i] = ""
        
test = 988
print("%".join(lines[test-10:test+10]))
for i,line in enumerate(lines):
    if(isAction(i,lines)):
    #    print(i,line)
        del lines[i:i+4]
    #if i > 1000:
     #   break
for i,line in enumerate(lines):
    txtfile.write(line+"\n")

for i,line in enumerate(lines):
    if('(' in line):
        if("CONT'D" not in line and "O.S." not in line and "V.O" not in line and "MORE" not in line):
            pre_emotion_empty_count = 0
            pre_step = 0
            pre_name_count = 0
            result = []
        
            while(pre_name_count < 1 or len(lines[i - pre_step]) < 3 or  not  isAllCaps(lines[i-pre_step])):
                if isAllCaps(lines[i-pre_step]) and len(lines[i-pre_step]) > 0:
                    pre_name_count += 1
                pre_step += 1
            if("INT." in lines[i-pre_step] or "EXT." in lines[i-pre_step]):
                result.append("")
                result.append("")
                result.append("")
                pre_step-=1
                while(lines[i-pre_step] == ""):
                    pre_step-=1
                
            post_emotion_empty_count = 0
            post_step = 0
            post_name_count = 0

            while(post_name_count < 2 or len(lines[i + post_step]) < 3 or  not  isAllCaps(lines[i+post_step]) ):
                if "EXT." in lines[i+post_step] or "INT." in lines[i+post_step] :
                    break
                if isAllCaps(lines[i+post_step]) and len(lines[i+post_step]) > 0:
                    post_name_count += 1
                post_step += 1
            while "EXT." in lines[i+post_step] or "INT." in lines[i+post_step]:
                post_step -=1
            
            #print(lines[i-pre_step:i+post_step])
            #print(pre_step,post_step)
            printed_lines = lines[i-pre_step:i+post_step]
            
            for line_in_range in lines[i-pre_step:i+post_step]:
                if(line_in_range!="" and not line_in_range[:-1].isnumeric()):
                    if(line_in_range!="(O.S.)"):
                        if(len(result)%3 == 1 and result[-1][0] != '(' and line_in_range[0]!='('):
                            result.append("")
                        result.append(line_in_range)
            name_utterance = []
            tmp_utterance = "%%%"
            for x in result:
                if(isName(x)):
                    if(tmp_utterance != "%%%"):
                        name_utterance.append(tmp_utterance)
                    tmp_utterance = ""
                    name_utterance.append(x)
                else:
                    tmp_utterance += x
                
            if(tmp_utterance != ""):
                name_utterance.append(tmp_utterance)
            #print("=".join(name_utterance))
            out.write("\t".join(name_utterance)+"\n")
            #print("+".join(result))
            #out.write("\t".join(result)+"\n")
            #print(discovery,i)
            discovery+=1
            #if(discovery>50):
             #   break
    elif(line.count("(")>2):
        print("Line {0} has more than one (",i)

['BOYHOOD ', '', ' ', '', 'Written by Richard Linklater', '', 'Boyhood Inc. ', '1901 E. 51st Street ', 'Austin, TX 78723 ', '512.322.0031', '', '\x0cEXT.  OUTSIDE SCHOOL - DAY', '', 'MASON waits outside his elementary school, laying in the', 'grass staring up at the clouds. In the distance, the school', "door opens and Mason's MOM, Olivia, walks toward him.   ", '', 'Hey, love bug.  You ready?', '', 'MOM', '', 'He is quickly on his feet and they are walking toward the', 'car.', '', 'Yeah.  Hey, guess what, Mom?', '', 'MASON', '', 'What?', '', 'MOM', '', 'I figured out where wasps come from.', '', 'MASON', '', 'Oh, yeah?  Where?', '', 'MOM', '', 'MASON', '', 'Well, I think it must be if you flick', "a rock into the air just right, it'll", 'turn into a wasp.', '', 'Dang...', '', 'Yep.', '', "That's cool.", '', 'MOM', '', 'MASON', '', 'MOM', '', 'INT. CAR - DAY', '', 'Driving...', '', 'So how was your day at school?', '', 'MOM', '', 'Fine.', '', 'MASON', '', 'MOM', '', 'Hey, I had a good 

AttributeError: 'str' object has no attribute 'isnumeric'